In [2]:
import numpy as np
import math
import torch
import torch.nn as nn

In [13]:
a1 = torch.tensor(np.pi)
a2 = torch.tensor(math.pi)



torch.float32


In [17]:
p = 3


b1 = torch.log(torch.tensor(2*np.pi**(p/2)))
b2 = torch.log(torch.tensor(3/4*np.pi))
print(b1,b2)

tensor(2.4102) tensor(0.8570)


In [78]:
class WatsonDistribution(nn.Module):
    """
    Multivariate Watson distribution class
    """
    def __init__(self,p, para_init=None):
        super().__init__()
        self.p = p
        self.mu = nn.Parameter(torch.ones(3))
        self.kappa = nn.Parameter(torch.rand(self.p,self.p))
        

W = WatsonDistribution(p=3)

In [82]:
W.state_dict()['mu'] *= 3

In [83]:
W.state_dict()

OrderedDict([('mu', tensor([3., 3., 3.])),
             ('kappa',
              tensor([[0.2341, 0.1961, 0.5224],
                      [0.1311, 0.9199, 0.1865],
                      [0.9313, 0.2443, 0.9681]]))])

In [56]:
x = torch.tensor([3,3,3])
A = torch.tensor([[1,1,1],[2,2,2],[10,10,10]])
torch.matmul(x,A)

tensor([39, 39, 39])

In [74]:
mu = nn.Parameter(torch.ones(3,3))
mu.__repr__

<bound method Parameter.__repr__ of Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)>

In [90]:
K = 10
p=3
mu = torch.zeros(p,K)
for j in range(K):
        val = 1 if j % 2 == 0 else -1
        mu[(j*int(p/K)),j] = val
mu

tensor([[ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [92]:
torchtorch.ones(K)*1/K

tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000])